# Confirm Weirdness
* In `Demo.ipynb` we see GPT4.1 repeatedly failing to genrate 400 sentences of pair `nl-da`
* However, there were instances where it DID work using the same prompt.

In [1]:
from scripts.translators import GPTClient
gpt = GPTClient()
print(gpt.user_prompt('nl', 'da', 'text'))

Translate the following Dutch sentences into Danish.
Please make sure to keep the same formatting, do not add more newlines.
Here is the text:
text


* The above prompt was used in `Demo.ipynb` as well.
* The commit history of this branch and the logged commit hash in `log.jsonl` can prove this.

In [2]:
from scripts.data_management import EuroParlManager
from scripts.util import split_sents
dm = EuroParlManager()
nl_sents, da_sents = dm.get_sentence_pairs('nl', 'da', 400)
print(f'Sent Length by Dataset: nl: {len(nl_sents)}, da: {len(da_sents)}')
nl_sents_split, da_sents_split = split_sents('\n'.join(nl_sents), 'nl'), split_sents('\n'.join(da_sents), 'da')
print(f'Sent Length after Split: nl: {len(nl_sents_split)} da: {len(da_sents_split)}')

Sent Length by Dataset: nl: 400, da: 400
Sent Length after Split: nl: 460 da: 417


In [3]:
for _ in range(10):
    mt_sents = gpt.translate_document(
        text=nl_sents,
        src_lang='nl',
        tgt_lang='da'
    )
    print(len(mt_sents))
    mt_sents_nl = split_sents('\n'.join(mt_sents), 'nl')
    mt_sents_da = split_sents('\n'.join(mt_sents), 'da')
    print(len(mt_sents_nl))
    print(len(mt_sents_da))
    if len(mt_sents) == 400: break

126
169
153
125
168
152
126
169
153
400
506
467


In [4]:
from scripts.scoring import compute_bleu
compute_bleu(da_sents, mt_sents)

28.800967220431154

In [5]:
compute_bleu(nl_sents, mt_sents)

1.9156893987883605

* Re-running DOES work but it feels random
* Running it 10 times can be in some cases not enough, as seen in `Demo.ipynb`